# Gaussian

In [1]:
#Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
#Import to dataframe
df_features = pd.read_csv('processed_hotel_bookings.csv')

In [3]:
#The dataset is quite large, and I am getting memory errors on this particular algorithm.
#I will split it randomnly.
df_features = df_features.sample(frac=0.05, random_state=763)

#We need to reset the index so the the algorithms below can loop through the dataframe
df_features = df_features.reset_index()

### Data Prep with t-SNE dimensionality reduction

Elaboration of my dimensionality reduction configuration decision process is given in the Final Dimension Reduction Configuration notebook.

In [4]:
#Libraries
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize

In [5]:
# Loading the data
X = df_features
y = df_features.is_canceled

# Standarizing the features
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

X_norm = normalize(X_std)

In [6]:
from sklearn.manifold import TSNE
import time

In [7]:
time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=100, n_iter=300)
tsne_results = tsne.fit_transform(X_std)

print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

[t-SNE] Computing 301 nearest neighbors...
[t-SNE] Indexed 1978 samples in 0.016s...
[t-SNE] Computed neighbors for 1978 samples in 0.296s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1978
[t-SNE] Computed conditional probabilities for sample 1978 / 1978
[t-SNE] Mean sigma: 1.125884
[t-SNE] KL divergence after 250 iterations with early exaggeration: 57.463097
[t-SNE] KL divergence after 300 iterations: 0.498681
t-SNE done! Time elapsed: 8.447791576385498 seconds


## Gaussian

I noted that Gaussian does better on the data before dimensionality reduction than after.

In [8]:
#Libraries
from sklearn.mixture import GaussianMixture
from matplotlib.patches import Ellipse

In [9]:
#Libraries
from sklearn import datasets
from sklearn import metrics

### The Best Performer

The Best Performing Algorithm was with 2 clusters and diagonal covariance

In [10]:
# Defining the agglomerative clustering
gmm_cluster = GaussianMixture(n_components=2, random_state=123, covariance_type='diag')
_
# Fit model
G_clusters = gmm_cluster.fit_predict(X_std)

In [11]:
plt.figure(figsize=(10,5))
colours = 'rbg'
for i in range(tsne_results.shape[0]):
    plt.text(tsne_results[i, 0], tsne_results[i, 1], str(G_clusters[i]),
             color=colours[y[i]],
             fontdict={'weight': 'bold', 'size': 50}
        )

plt.xticks([])
plt.yticks([])
plt.axis('off')
plt.show()

In [14]:
G_clusters = np.array([0 if x == 1 else 1 if x==0 else 2 for x in G_clusters])

pd.crosstab(y, G_clusters)

col_0,0,1
is_canceled,,
0,12,1454
1,512,0


### Finding the Best Performer

In [12]:
# Gaussian Mixture takes in covariance type, range, x, and y
def gmm_ari_silh(covariance_type, range_num, x, y):
  from sklearn.mixture import GaussianMixture  
  for num in range(2, range_num, 1):
    pred = GaussianMixture(n_components=num, covariance_type=covariance_type).fit_predict(x)
    print("ARI score with {} clusters and covariance_type={}: {}".format(num, covariance_type,
      metrics.adjusted_rand_score(y, pred)))

    print("Silhouette score with {} clusters and covariance_type={}: {}".format(num, covariance_type,
      metrics.silhouette_score(x, pred, metric='euclidean')))
    print('='*90)

In [13]:
# check Gaussian Mixture clustering algorithm
types = ['full', 'tied', 'diag', 'spherical']
for i in types:
  gmm_ari_silh(i, 7, X_std, y)
  print('\n')

ARI score with 2 clusters and covariance_type=full: 0.9516440359097267
Silhouette score with 2 clusters and covariance_type=full: 0.2477974332641078
ARI score with 3 clusters and covariance_type=full: 0.3694525651801217
Silhouette score with 3 clusters and covariance_type=full: 0.21515257954551098
ARI score with 4 clusters and covariance_type=full: 0.2739504032812048
Silhouette score with 4 clusters and covariance_type=full: 0.22999356988410724
ARI score with 5 clusters and covariance_type=full: 0.09054348004462516
Silhouette score with 5 clusters and covariance_type=full: 0.17739886867053845
ARI score with 6 clusters and covariance_type=full: 0.29006730485561566
Silhouette score with 6 clusters and covariance_type=full: 0.19429699686982785


ARI score with 2 clusters and covariance_type=tied: 0.011254234239765435
Silhouette score with 2 clusters and covariance_type=tied: 0.19020032205447535
ARI score with 3 clusters and covariance_type=tied: 0.38380492714394965
Silhouette score with 3